In [28]:
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
import sklearn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [29]:
base = pd.read_csv('datasets/petr4_treinamento.csv')
base

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,19.990000,20.209999,19.690001,19.690001,18.086271,30182600.0
1,2013-01-03,19.809999,20.400000,19.700001,20.400000,18.738441,30552600.0
2,2013-01-04,20.330000,20.620001,20.170000,20.430000,18.766001,36141000.0
3,2013-01-07,20.480000,20.670000,19.950001,20.080000,18.444506,28069600.0
4,2013-01-08,20.110001,20.230000,19.459999,19.500000,17.911745,29091300.0
...,...,...,...,...,...,...,...
1240,2017-12-25,15.750000,15.750000,15.750000,15.750000,15.718563,0.0
1241,2017-12-26,15.750000,15.990000,15.690000,15.970000,15.938125,22173100.0
1242,2017-12-27,15.990000,16.139999,15.980000,16.049999,16.017963,23552200.0
1243,2017-12-28,16.100000,16.129999,16.000000,16.100000,16.067865,19011500.0


In [30]:
base.isnull().sum()

Date         0
Open         3
High         3
Low          3
Close        3
Adj Close    3
Volume       3
dtype: int64

In [31]:
base = base.dropna()

In [32]:
base.shape

(1242, 7)

In [33]:
base_treinamento = base.iloc[:, 1:2].values #apenas a primeira coluna(abertura)

In [34]:
base_treinamento

array([[19.99    ],
       [19.809999],
       [20.33    ],
       ...,
       [15.99    ],
       [16.1     ],
       [16.1     ]])

In [35]:
normalizador = MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)
base_treinamento_normalizada

array([[0.76501938],
       [0.7562984 ],
       [0.78149225],
       ...,
       [0.57122093],
       [0.57655039],
       [0.57655039]])

In [36]:
base_treinamento_normalizada.shape

(1242, 1)

In [37]:
X = [] #previsores
y = [] #preço real
for i in range(90, 1242): #90 preços anteriores
    X.append(base_treinamento_normalizada[i-90: i, 0])
    y.append(base_treinamento_normalizada[i,0])
    
X, y = np.array(X), np.array(y)

In [38]:
X.shape

(1152, 90)

In [39]:
y.shape

(1152,)

In [40]:
regressor = Sequential()

regressor.add(LSTM(units = 100, return_sequences=True, input_shape=(X.shape[1], 1)))#x.shape[1],1 enviando 90 colunas para a rede neural, apenas a coluna com valor de abertura
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(Dense(units=1, activation='linear'))

regressor.summary()

c:\Users\andre\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 90, 100)        │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 90, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 90, 50)         │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 90, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 90, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 90, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 90, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 90, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 90, 1)          │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,451 (435.36 KB)

 Trainable params: 111,451 (435.36 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
regressor.compile(optimizer='rmsprop', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])

In [42]:
regressor.fit(X, y, epochs=100, batch_size=32)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - loss: 0.1032 - mean_absolute_error: 0.2601
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 0.0518 - mean_absolute_error: 0.1869
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 0.0461 - mean_absolute_error: 0.1736
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 0.0465 - mean_absolute_error: 0.1775
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.0458 - mean_absolute_error: 0.1750
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.0408 - mean_absolute_error: 0.1654
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.0426 - mean_absolute_error: 0.1693
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.0435 - mean_absolute_error: 0.1691
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.0411 - mean_absolute_error: 0.1663
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.0401 - mean_absolute_error: 0.1625
Epoch 11/100
36/36 ━━━━━━━━━━